In [2]:
import pickle 
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv("/content/drive/MyDrive/HamoyeInternship/Team_Algorithm/CVSS_metrics_prediction/cve_merged.csv")
df.head()

,cve_id,mod_date,pub_date,cvss,cwe_code,cwe_name,summary,access_authentication,access_complexity,access_vector,impact_availability,impact_confidentiality,impact_integrity,vendor,vulnerable_product,Pub_Year,Mod_Year,Qualitative_cvss
0,CVE-2019-2211,2019-11-14 21:36:00,2019-11-13 18:15:00,7.8,89,Improper Neutralization of Special Elements u...,In createProjectionMapForQuery of TvProvider.j...,NONE,LOW,NETWORK,NONE,COMPLETE,NONE,google,android,2019,2019,High
1,CVE-2019-2212,2019-11-14 21:30:00,2019-11-13 18:15:00,4.9,200,Information Exposure,"In poisson_distribution of random, there is an...",NONE,LOW,LOCAL,NONE,COMPLETE,NONE,google,android,2019,2019,Medium
2,CVE-2019-2213,2019-11-14 21:24:00,2019-11-13 18:15:00,6.9,416,Use After Free,"In binder_free_transaction of binder.c, there ...",NONE,MEDIUM,LOCAL,COMPLETE,COMPLETE,COMPLETE,google,android,2019,2019,Medium
3,CVE-2019-2214,2019-11-14 21:19:00,2019-11-13 18:15:00,7.2,269,Improper Privilege Management,"In binder_transaction of binder.c, there is a ...",NONE,LOW,LOCAL,COMPLETE,COMPLETE,COMPLETE,google,android,2019,2019,High
4,CVE-2019-18793,2019-11-14 21:14:00,2019-11-13 20:15:00,4.3,79,Improper Neutralization of Input During Web P...,Parallels Plesk Panel 9.5 allows XSS in target...,NONE,MEDIUM,NETWORK,NONE,NONE,PARTIAL,parallels,parallels_plesk_panel,2019,2019,Medium


In [ ]:
df["summary"][34]

'A stored cross-site scripting (XSS) vulnerability exists in Magento 2.2 prior to 2.2.10, Magento 2.3 prior to 2.3.3 or 2.3.2-p1. An authenticated user can inject arbitrary JavaScript code into the attribute set name when listing the products.'

**Load models**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#Define f1_score function to load deep learning cvss metrics model
import keras.backend as K
from keras.callbacks import Callback,ModelCheckpoint,EarlyStopping
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [1]:
!pip install -U "tensorflow-text==2.8.*"
!pip install -U "tensorflow==2.8.*"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.7 MB/s 
     |████████████████████████████████| 497.9 MB 34 kB/s 
     |████████████████████████████████| 1.4 MB 47.9 MB/s 
     |████████████████████████████████| 462 kB 58.7 MB/s 
     |████████████████████████████████| 5.8 MB 48.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2

In [1]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf


In [8]:
#load cvss metrics model
from tensorflow.keras.models import load_model
path = "/content/drive/MyDrive/HamoyeInternship/Team_Algorithm/bert_multi_ouput_all.h5"
model = load_model(
       (path),
       custom_objects={'KerasLayer':hub.KerasLayer,"f1_score":f1_score})
#load cvss score model
with open('/content/drive/MyDrive/HamoyeInternship/Team_Algorithm/CVSS_prediction/cvss_model.pkl', 'rb') as handle:
    model_cvss = pickle.load(handle)

**Custom prediction function**

In [9]:
! pip install streamlit -q

     |████████████████████████████████| 9.2 MB 7.6 MB/s 
     |████████████████████████████████| 182 kB 53.1 MB/s 
     |████████████████████████████████| 4.7 MB 45.5 MB/s 
     |████████████████████████████████| 78 kB 6.8 MB/s 
     |████████████████████████████████| 237 kB 55.5 MB/s 
     |████████████████████████████████| 164 kB 52.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 


In [10]:
%%writefile app.py
import pickle 
import numpy as np
import pandas as pd
import streamlit as st

#Define f1_score function to load deep learning cvss metrics model
import keras.backend as K
from keras.callbacks import Callback,ModelCheckpoint,EarlyStopping
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf

#load cvss metrics model
from tensorflow.keras.models import load_model
path = "/content/drive/MyDrive/HamoyeInternship/Team_Algorithm/bert_multi_ouput_all.h5"
model = load_model(
       (path),
       custom_objects={'KerasLayer':hub.KerasLayer,"f1_score":f1_score})
#load cvss score model
with open('/content/drive/MyDrive/HamoyeInternship/Team_Algorithm/CVSS_prediction/cvss_model.pkl', 'rb') as handle:
    model_cvss = pickle.load(handle)

target_dic = {'access_authentication':['MULTIPLE', 'NONE', 'SINGLE'],
 'access_vector': ['ADJACENT_NETWORK', 'LOCAL', 'NETWORK'],
 'access_complexity':['HIGH', 'LOW', 'MEDIUM'],
 'impact_availability':['COMPLETE', 'NONE', 'PARTIAL'],
 'impact_confidentiality':['COMPLETE', 'NONE', 'PARTIAL'],
 'impact_integrity':['COMPLETE', 'NONE', 'PARTIAL']}

def predict(Input_):
  input_text = [Input_]
  preds1 =  model.predict(input_text) #predict cvss metric
  class_index= []
  for i in  preds1:
    index = np.argmax(i)
    class_index.append(index)
  target_array = list(target_dic.values())
  targets = []
  for i,j in zip(target_array,class_index):
    val = i[j]
    targets.append(val)
  prediction_metric_cvss = {} # store as a dictionary
  for feature,label in zip(list(target_dic.keys()),targets):
    prediction_metric_cvss[feature] = label
  #one hot encode predicted metrics
  x = x = pd.DataFrame(prediction_metric_cvss, index=range(1))

  x = pd.get_dummies(data=x[['access_authentication', 'access_vector', 'access_complexity', 
                    'impact_availability', 'impact_confidentiality', 'impact_integrity']], 
                    prefix=['access_authentication', 'access_vector', 'access_complexity', 'impact_availability',
                            'impact_confidentiality', 'impact_integrity'],)
  predicted_cols = x.columns
  feature_cols = ['access_authentication_MULTIPLE', 'access_authentication_NONE',
       'access_authentication_SINGLE', 'access_vector_ADJACENT_NETWORK',
       'access_vector_LOCAL', 'access_vector_NETWORK',
       'access_complexity_HIGH', 'access_complexity_LOW',
       'access_complexity_MEDIUM', 'impact_availability_COMPLETE',
       'impact_availability_NONE', 'impact_availability_PARTIAL',
       'impact_confidentiality_COMPLETE', 'impact_confidentiality_NONE',
       'impact_confidentiality_PARTIAL', 'impact_integrity_COMPLETE',
       'impact_integrity_NONE', 'impact_integrity_PARTIAL']
  dict_cvss_input = {}
  for f in feature_cols:
    if f not in predicted_cols:
      dict_cvss_input[f] = 0
    else:
      dict_cvss_input[f] = 1
  input_data =  pd.DataFrame(dict_cvss_input,index=range(1))
  cvss_score = np.round(model_cvss.predict(input_data),2)
  return prediction_metric_cvss,cvss_score

def main():
    # Face Analysis Application #
    st.title("Prediction of CVSS metrics and cvss score via vulnerabiity text description ")
    activities = ["Home", "Predict metrics and cvss score", "About"]
    choice = st.sidebar.selectbox("Select Activity", activities)
    st.sidebar.markdown(
        """ Developed by Team algorithm for the hamoye 2022 summer capstone project  
            Email us @ : owususammy509@gmail.com
            """)
    if choice == "Home":
        html_temp_home1 = """<div style="background-color:#6D7B8D;padding:10px">
                                            <h4 style="color:white;text-align:center;">
                                            Definition :Most sofware vulnerabiities are not exploited so there is need for companies to provide remedy to high priority vulnerabiity first and promptly.</h4>
                                            </div>
                                            </br>"""
        st.markdown(html_temp_home1, unsafe_allow_html=True)
        st.write("""The web app helps software companies to predict cvss (common vulnerability scoring systems) baseline metrics and the score.The output generated are:
                 """)

        st.write("""1.The cvss metrics""")
        st.write("""2.The cvss score""")
        st.write("""3.The qualitative cvss score""")

    elif choice == "Predict metrics and cvss score":
            input_text = st.text_input("Give a technical summary of the vulnerabiity")
            x = str(input_text)

          
            
          
            if st.button("Predict"):
                Metrics,cvss_score = predict(x)
                Metrics = list(Metrics.values())
                st.markdown("Below are the cvss baseline metrics")

                st.write("**Access Authentication:**",*Metrics[0])
                st.write("**Access Vector:**",Metrics[1])
                st.write("**Access Complexity:**",Metrics[2])
                st.write("**Impact Availability:**",Metrics[3])
                st.write("**Impact Confidentiality:**",Metrics[4])
                st.write("**Impact Integrity:**",Metrics[5])
                st.write("___________________________________")
                score = cvss_score[0]
                st.write("**Generated CVSS score is:**",score)
                st.write("____________________________________")
                if score  == 0.0:
                    st.write("**Severity level of vulnerabiity**:None")
                elif 0.1 <= score <= 3.9:
                    st.write("**Severity level of vulnerabiity**:Low")
                elif 4.0 <= score <= 6.9:
                    st.write("**Severity level of vulnerabiity**:Medium")
                elif 7.0 <= score <= 8.9:
                    st.write("**Severity level of vulnerabiity**:High")
                else:
                    st.write("**Severity level of vulnerabiity**:Critical")

                

              



                    
                             
                      

                                  
                
            
            
            
            
            
            
            
            

    elif choice == "About":
        st.subheader("About this app")
        html_temp_about1= """<div style="background-color:#6D7B8D;padding:10px">
                                    <h4 style="color:white;text-align:center;">
                                    Definition : </h4>
                                    </div>
                                    </br>"""
        st.markdown(html_temp_about1, unsafe_allow_html=True)

        html_temp4 = """
                                     <div style="background-color:#98AFC7;padding:10px">
                                     <h4 style="color:white;text-align:center;">This Application is developed by Team algorithm of hamoye summer internshop 2022 using Streamlit Framework for the purpose of desmontrating softaware vulnerabiities severity precision</h4>
                                     <h4 style="color:white;text-align:center;">Thanks for Visiting</h4>
                                     </div>
                                     <br></br>
                                     <br></br>"""

        st.markdown(html_temp4, unsafe_allow_html=True)

    else:
             pass


        


if __name__ == "__main__":
    main()

    

Writing app.py


In [11]:
! pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 4.5 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=b26ad59812ea21d2c54eabeba8321f2cb0d80d275a5e333b2a77bf28b162f09f
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [12]:
from pyngrok import ngrok

In [13]:
ngrok.set_auth_token("2GQD6b7qkrEjKa9cKV6aIXOr4oW_6c7VdXeuqjS6N7iUYMYvG") #ngrok.com

In [14]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "http://8215-34-125-65-29.ngrok.io" -> "http://localhost:80"


In [15]:
ngrok.connect(port = '8080')

PyngrokNgrokHTTPError: ignored